# Q-learning notebook

Reinforcement learning (RL) is a fundamental branch of machine learning where an agent learns to make optimal decisions by interacting with an environment. This notebook focuses on action selection strategies in a reinforcement learning context, specifically within the Frozen Lake environment.

## Objective
Our primary goal is to implement and understand the Q-learning algorithm. We will create an action selection function that determines the next move of an agent based on the following approach:

Exploration: The agent selects a random action to gather more knowledge about the environment.
Exploitation: The agent chooses the action that maximizes its expected future rewards based on prior experience (Q-values).
By implementing and fine-tuning this strategy, we aim to enhance the agent’s ability to navigate efficiently through the environment.

## Key Takeaways

* Understanding the balance between exploration and exploitation.
* Implementing ε-greedy action selection.
* Leveraging a Q-table to improve decision-making.
* Applying concepts to a real reinforcement learning problem using OpenAI Gym’s Frozen Lake environment.

Now, let’s dive in and start implementing our action selection strategy!

In [ ]:
import numpy as np
import random

In [ ]:
# Define the environment as a grid map
# 'S' = Start, 'F' = Frozen surface, 'H' = Hole (fall in = lose), 'G' = Goal (reach = win)
env_map = [
    ['S', 'F', 'F', 'F'],
    ['F', 'H', 'F', 'H'],
    ['F', 'F', 'F', 'H'],
    ['H', 'F', 'F', 'G']
]


# Total number of states (each cell in the grid is a state)
# len(env_map) -> number of rows
# len(env_map[0]) -> number of columns
num_states = len(env_map) * len(env_map[0])

# Number of possible actions (Up, Down, Left, Right)
num_actions = 4

# Whether the environment is slippery (affects movement randomness)
is_slippery = False  # If False, movement is deterministic; if True, the agent can slip

# Maximum exploration rate (initially the agent explores a lot)
max_exploration_rate = 1.0

# Minimum exploration rate (ensures some exploration even after training)
min_exploration_rate = 0.01

# Initialize the Q-table with zeros
# This table stores the learned Q-values for each (state, action) pair
q_table = np.zeros((num_states, num_actions))

# Start at the initial state (usually the top-left corner of the grid)
state = 0  # This corresponds to the 'S' position in the grid

# Task 0: Implementing the Action Selection Strategy

In this task, we will create a function `get_next_action` that determines the next action our agent should take in the **Frozen Lake** environment. The action selection is based on the **exploration vs. exploitation** strategy:

1. **Generate** a random number and compare it with the `exploration_rate`.
2. If the random number is **less than** the exploration rate, the agent will **explore** by selecting a random action.
3. Otherwise, the agent will **exploit** the knowledge from the Q-table by choosing the action with the highest Q-value.

### Instructions:

In the code cell below, fill in the two lines marked with `# TODO`:

- **One line** to select a random action (exploration).
- **One line** to select the best-known action (exploitation).

### Helpful Hints:

- Select a random action from the available actions using `random.choice(...)`. Since the Q-table's shape is `[number_of_states, number_of_actions]`, the number of available actions is given by `q_table.shape[1]`.
- `random.choice(range(10))` produces random numbers from 0 to 9 (including).
- Find the best action using `np.argmax(...)` on the Q-values for the current state. Remember, the shape of `q_table` is `[number_of_states, number_of_actions]`. Access the Q-values for all actions in the current state using `q_table[state]`.


In [ ]:
def get_next_action(q_table, state, exploration_rate):
    """
    Choose the next action using an epsilon-greedy exploration/exploitation strategy.

    Parameters:
    - q_table (list): Q-values table with shape (num_states, num_actions).
    - state (int): Current state index.
    - exploration_rate (float): Probability of choosing a random action (epsilon).

    Returns:
    - action (int): Selected action index.
    """
    # Compare a random number with the exploration_rate
    if random.uniform(0, 1) < exploration_rate:
        # Explore: choose a random action
        # TODO: Fill the next line to choose a random action (hint: random.choice or random.randint)
        action = ...
    else:
        # Exploit: choose the best action based on Q-values
        # TODO: Fill the next line to choose the action with the highest Q-value (hint: np.argmax)
        action = ...

    return action

# Task 1: Implementing the Q-learning Update Rule

In this task, we will update our Q-table using the **Q-learning** update formula after selecting an action and observing the next state and reward:

$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left(r + \gamma \max_{a'} Q(s', a') - Q(s, a)\right)
$$

where

- **$Q(s, a)$**: Current Q-value for state $s$ and action $a$.
- **$\alpha$ (learning rate)**: Controls how quickly the agent learns.
- **$\gamma$ (discount factor)**: Determines how much future rewards are valued.
- **$r$**: Immediate reward received after taking the action.
- **$\max_{a'} Q(s', a')$**: Maximum Q-value among all possible actions in the next state $s'$.


The update process involves the following steps:

1. **Select** an action using the provided `get_next_action(state, exploration_rate)` function.
2. **Perform** the selected action and observe the new state and the immediate reward.
3. **Update** the Q-value for the state-action pair based on the observed reward and the estimated optimal future reward.
4. **Return** the new state, the reward and indicate if the episode has ended.

### Instructions:

In the code cell below, complete the implementation by filling in the line marked with `# TODO`:

- **One line** to update the Q-table entry for the current state-action pair using the Q-learning formula.

### Helpful Hints:

- Retrieve the maximum Q-value of the next state using `np.max(q_table[new_state])`.
- Ensure the Q-table update exactly follows the provided formula, correctly applying the learning rate (`alpha`) and discount factor (`gamma`).


### Understanding `get_next_state`

The `get_next_state` function calculates the next state of the agent based on its current state and chosen action. It returns the actual action taken due to the fact that slippery environment may change the given action, the next state, the reward obtained (1 if the goal is reached, 0 otherwise), and a boolean indicating whether the episode has ended due to reaching a goal (`'G'`) or falling into a hole (`'H'`).

In [ ]:
from helper_functions import get_next_state

def q_learning_execute_one_step(q_table, state, exploration_rate, learning_rate, discount_factor):
    """
    Execute a single step of the Q-learning algorithm:

    Parameters:
    - q_table (list): Table containing Q-values for state-action pairs.
    - state (int): Current state of the agent.
    - exploration_rate (float): Probability of choosing a random action (exploration vs. exploitation).
    - learning_rate (float): Rate at which the Q-values are updated.
    - discount_factor (float): How much future rewards are valued.

    Returns:
    - new_state (int): The next state after executing the action.
    - reward (int): Immediate reward after taking the action at given state
    - done (bool): Flag indicating whether the episode has ended (i.e., hitting on a hole or present).
    """
    # Step 1: Choose an action
    action = get_next_action(q_table, state, exploration_rate)

    # Step 2: Execute the action in the environment
    # Warning: Here, get_next_state(...) may return different action than the given one if the environment is slippery.
    action, new_state, reward, done = get_next_state(state, action)

    # Step 3: Update the Q-table
    # TODO: Fill in the next line using the Q-learning formula (line 10)
    q_table[state, action] = ...

    # Step 4: Return the new state and done flag
    return new_state, reward, done

# Task 2: Running One Episode of Q-Learning

In this task, we will run a full episode of **Q-learning**, meaning the agent will interact with the environment for multiple steps until the episode ends or the maximum number of steps is reached.

### Each episode follows this process:
1. **Start** from the given state.
2. **Repeat** for a fixed number of steps (or until the episode ends):
   - Take one step of Q-learning.
   - **Accumulate** the total reward, and update the current state.
   - **Stop** if the episode has ended.
3. **Return** the total reward earned in the episode.

## Instructions:

Complete the missing part inside the `for` loop in the function below. You need to:

- **Call** `q_learning_execute_one_step(...)` to execute one step of Q-learning.
- **Update** `total_reward` with the received reward.
- **Update** `state` with the new state.
- **Check** if `done` is `True`, **stop** the loop.

In [ ]:
def q_learning_execute_one_episode(q_table, state, max_steps, exploration_rate, learning_rate, discount_factor):
    """
    Run one episode of Q-learning.

    Parameters:
    - q_table (list): Table containing Q-values for state-action pairs.
    - state (int): Initial state of the agent.
    - max_steps (int): Maximum number of steps per episode.
    - exploration_rate (float): Probability of choosing a random action (exploration).
    - learning_rate (float): Rate at which the Q-values are updated.
    - discount_factor (float): How much future rewards are valued.

    Returns:
    - total_reward (int): The sum of all rewards received in the episode.
    """
    done = False
    total_reward = 0
    for step in range(max_steps):
        # TODO: Call the function that executes one step of Q-learning and update variables
        new_state, reward, done = ...
        total_reward = ...
        state = ...

        # TODO: Check if the episode has ended
        if ...:
            break

    return total_reward

# Task 3: Running Multiple Episodes of Q-Learning with Exploration Rate Decay  

In this task, we will run the **Q-learning algorithm** for multiple episodes, allowing the agent to explore and improve its decision-making over time.  

### The Q-learning process follows these steps:
1. **Loop over multiple episodes** to allow the agent to learn over time.
2. **In each episode**:
   - Run one full episode of Q-learning.
   - Store the **total reward** received in the episode.
   - Adjust the **exploration rate** using an exponential decay function.
3. **Return** the final exploration rate after all episodes.


## Exploration Rate Decay  

In **Q-learning**, the agent must balance between:  
- **Exploration** (trying new actions to discover better rewards).  
- **Exploitation** (choosing the best-known action based on learned values).  

To **control this balance**, we gradually **decrease the exploration rate** over time, encouraging the agent to exploit learned knowledge in later episodes. We use the following decay function to adjust the **exploration rate**:

$$
\text{exploration\_rate} = \text{min\_exploration\_rate} + (\text{max\_exploration\_rate} - \text{min\_exploration\_rate}) \times e^{-\text{decay\_rate} \times \text{episode}}
$$

### Parameters:
- **`min_exploration_rate`**: The minimum exploration rate (ensures the agent always explores a little).
- **`max_exploration_rate`**: The initial exploration rate (starts high for more exploration).
- **`decay_rate`**: Controls how quickly the exploration rate decreases.
- **`episode`**: The current episode number.


<div style="text-align: center;">
    <figure>
        <img src="./assets/exploration_rate_decay.png" alt="Exploration Rate Decay" width="600">
        <figcaption><b>Figure 1:</b> Exploration rate decay over episodes (decay rate = 0.001)</figcaption>
    </figure>
</div>


## Instructions:

Complete the missing parts in the function below. You need to:
- **Call** `q_learning_execute_one_episode(...)` to execute one episode of Q-learning.
- **Update** the `exploration_rate` to decrease over time using the given formula.


In [ ]:
def q_learning(q_table, state, num_episodes, max_steps, learning_rate, discount_factor, exploration_rate, decay_rate):
    """
    Run the Q-learning algorithm for a specified number of episodes.

    Parameters:
    - q_table (list): Table containing Q-values for state-action pairs.
    - state (int): Initial state of the agent.
    - num_episodes (int): Total number of episodes to run.
    - max_steps (int): Maximum steps per episode.
    - learning_rate (float): Rate at which Q-values are updated.
    - discount_factor (float): How much future rewards are valued.
    - exploration_rate (float): Initial probability of choosing a random action.
    - decay_rate (float): Factor by which exploration_rate decreases over episodes.

    Returns:
    - exploration_rate (float): Updated exploration rate after all episodes.
    - episode_rewards
    """
    episode_rewards = []
    for episode in range(num_episodes):
        # TODO: Task 3, Call the function to execute one episode and store the reward
        total_reward = ...

        # TODO: Task 4, Update the exploration rate to gradually decrease
        # Do not forget the minus sign in np.exp(...).
        exploration_rate = ... + (... - ...) * np.exp(...)

        episode_rewards.append(total_reward)

    return exploration_rate, episode_rewards

## Playing the game

We now display the frozen lake game with the model that we trained. For this, run the cell below. Then click on ''run N episodes''. In the lower part of the window you can click on the button that displays the logs and then see how your program would work.

In [ ]:
%run -i helper_functions.py
run_game()